In [29]:
import os
import re
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

TRAIN_DATASET_PATH='D:/brats/nii/archive (1)/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'
surInfoFile = "D:/brats/1/BraTS2020_training_data/content/data/survival_info.csv"
MetaDataFile = "D:/brats/1/BraTS2020_training_data/content/data/meta_data.csv"

date_column = ['Brats20ID','Age','Survival_days','Extent_of_Resection']
df_surInfo = pd.read_csv(surInfoFile,names= date_column,skiprows=1)
df_metaData = pd.read_csv(MetaDataFile,names= ['slice_path','target','volume','slice'],skiprows=1)


def modify_value(value):
    idStr = str(value)
    number_str = idStr[-3:] 
    number_int = int(number_str)
    return number_int

def to_int(value):
    return int(value)


# Remove ids with issues 
## survival_days == Alive ##
df_surInfo = df_surInfo.drop(index=83) #id==84

## issue extracting original file ##
df_surInfo = df_surInfo.drop(index=98) #id==99
df_surInfo = df_surInfo.drop(index=225)#id==354

# Modify 
df_surInfo['Brats20ID'] = df_surInfo['Brats20ID'].apply(modify_value)
df_surInfo['Survival_days'] = df_surInfo['Survival_days'].apply(to_int)

In [30]:
from radiomics import featureextractor
import SimpleITK as sitk

def train_extractFeatures(train_ids):
    x_train = []
    y_train = []

    # Extractor set up 
    extractor = featureextractor.RadiomicsFeatureExtractor()
    print('Enabled filters:\n\t', extractor.enabledImagetypes)

    # Running through all patients
    count = 0
    for patient_dir in os.listdir(TRAIN_DATASET_PATH):

        # Getting patientId
        dirName = os.path.join(TRAIN_DATASET_PATH,patient_dir)
        patientId = re.search(r"Training_(\d+)", dirName).group(1)
        patient_id = int(patientId)  


        if patient_id in train_ids:
            featurelist = ['VoxelVolume', 'MeshVolume', 'SurfaceArea', 'SurfaceVolumeRatio',# 'Compactness1', 'Compactness2', 'SphericalDisproportion',
                            'Sphericity', 'Maximum3DDiameter','Maximum2DDiameterSlice','Maximum2DDiameterColumn','Maximum2DDiameterRow', 
                            'MajorAxisLength', 'MinorAxisLength', 'LeastAxisLength', 'Elongation', 'Flatness'] #14
            feature = []


            T1_path =  os.path.join(TRAIN_DATASET_PATH, patient_dir,patient_dir+ '_t1.nii')
            mask_path = os.path.join(TRAIN_DATASET_PATH, patient_dir,patient_dir+ '_seg.nii')
            print(str(count)+'/'+str(len(train_ids)))

            # Read .nii file
            T1_image = sitk.ReadImage(T1_path)
            mask = sitk.ReadImage(mask_path)


            age = df_surInfo.loc[df_surInfo['Brats20ID'] == patient_id, 'Age'].iloc[0]
            feature.append(age)

            # Extract features
            T1_feature_values = extractor.execute(T1_image, mask)


            for ft in featurelist:
                if 'original_shape_'+ft in  T1_feature_values:
                    value = T1_feature_values['original_shape_'+ft]
                    if isinstance(value, np.ndarray):
                        value = value[()]
                        value = value.astype(np.float64)
                        feature.append(value)
                    else:
                        feature.append(value)

            
            days = df_surInfo.loc[df_surInfo["Brats20ID"] == patient_id, 'Survival_days'].iloc[0]
            x_train.append(feature)
            y_train.append(days)
            count+=1
    
    return x_train,y_train




In [31]:
from radiomics import featureextractor
import SimpleITK as sitk

def test_extractFeatures(test_ids):
    x_test = []
    y_test = []

    # Extractor set up 
    extractor = featureextractor.RadiomicsFeatureExtractor()
    print('Enabled filters:\n\t', extractor.enabledImagetypes)

    # Running through all patients
    count = 0
    for patient_dir in os.listdir(TRAIN_DATASET_PATH):

        # Getting patientId
        dirName = os.path.join(TRAIN_DATASET_PATH,patient_dir)
        patientId = re.search(r"Training_(\d+)", dirName).group(1)
        patient_id = int(patientId)  


        if patient_id in test_ids:
            featurelist = ['VoxelVolume', 'MeshVolume', 'SurfaceArea', 'SurfaceVolumeRatio',# 'Compactness1', 'Compactness2', 'SphericalDisproportion',
                            'Sphericity', 'Maximum3DDiameter','Maximum2DDiameterSlice','Maximum2DDiameterColumn','Maximum2DDiameterRow', 
                            'MajorAxisLength', 'MinorAxisLength', 'LeastAxisLength', 'Elongation', 'Flatness'] #14
            feature = []


            T1_path =  os.path.join(TRAIN_DATASET_PATH, patient_dir,patient_dir+ '_t1.nii')
            mask_path = os.path.join(TRAIN_DATASET_PATH, patient_dir,patient_dir+ '_seg.nii')
            print(str(count)+'/'+str(len(test_ids)))

            # Read .nii file
            T1_image = sitk.ReadImage(T1_path)
            mask = sitk.ReadImage(mask_path)

            # Add age as feature
            age = df_surInfo.loc[df_surInfo['Brats20ID'] == patient_id, 'Age'].iloc[0]
            feature.append(age)

            # Extract features
            T1_feature_values = extractor.execute(T1_image, mask)


            for ft in featurelist:
                if 'original_shape_'+ft in  T1_feature_values:
                    value = T1_feature_values['original_shape_'+ft]
                    if isinstance(value, np.ndarray):
                        value = value[()]
                        value = value.astype(np.float64)
                        feature.append(value)
                    else:
                        feature.append(value)

            
            days = df_surInfo.loc[df_surInfo["Brats20ID"] == patient_id, 'Survival_days'].iloc[0]
            x_test.append(feature)
            y_test.append(days)
            count+=1
    
    return x_test,y_test


In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def RF_train_result(X_train,X_test,Y_train,Y_test,times):
    m = []
    for State in range(times):
        random_forest = RandomForestRegressor(n_estimators=10, random_state=State)  

        # Train the model
        random_forest.fit(X_train, Y_train)

        # Predict on the test set
        Y_pred = random_forest.predict(X_test)
        Y_pred = Y_pred.astype(int)
        mse = mean_squared_error(Y_test, Y_pred)

        # Calculate Mean Squared Error (MSE)
        m.append(mse)

    print(f"Average Mean Squared Error (MSE): {sum(m) / len(m)}")
    return sum(m) / len(m)


In [33]:
from sklearn.model_selection import train_test_split

result = []
for i in range(5):
    # Split ids
    all_ids = df_surInfo['Brats20ID'].unique()

    # Splitting the IDs into train and test sets (80-20 split) 
    train_ids, test_ids = train_test_split(all_ids, test_size=0.1489, random_state=i)

    print(f"Number of train IDs: {len(train_ids)}")
    print(f"Number of test IDs: {len(test_ids)}")
    print(train_ids)
    print(test_ids)

    X_train, Y_train =  train_extractFeatures(train_ids)
    X_test , Y_test  =  test_extractFeatures(test_ids)

    mse_result = RF_train_result(X_train,X_test,Y_train,Y_test,1)
    result.append(mse_result)




Number of train IDs: 198
Number of test IDs: 35
[109 159  19  16  56   8 177 196 181 175 100  91  75  34 125  64 254 223
 126 360 168  17 111  65  46 238  76  23 107 151   5 104  62 112  53  67
  27 163 252  41  14   4  94  25  31 129  61  57 342 110  20 339 179  55
 225  81  52 188 187 121  88 230 170 255 180  42  15  28  51 337  21 182
  47 186 363 158 222  63 184   3  60 183 348  44 221  11 242  74 356 359
 115 253 199 101  95 162 354 116   1  96  97  70  50  49  87 367 165  24
 167  79 103 153 226 368   7  69  86 124 346 215 247 241 217  93 240  12
 122 105  36  58  66   2 123 227  43 108 154 246  18  39 157  54 191 218
 147  35  29 117 190 178  32 229 145 224 251  33 195 166 235 173  30 216
 102  83 366 214  80 239 118 174 358  73  78  26 192  82 347 210 353  40
 233  59 164  90 345  71  89  37  22 338  10 106 236  68 231 120  48 204]
[351 169 206 228 160 152 250 201  92 185 161 127 113 119 203   9  72 340
 114   6 220  45 369  98 189 249  38  77 219 341  13 128  85 194 357]
Enabl

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


1/198


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


2/198


...

33/35


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


34/35


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Average Mean Squared Error (MSE): 68678.25714285714


In [34]:
for i in result:
   print(f"Mean Squared Error (MSE): {i}")

Mean Squared Error (MSE): 132188.25714285715
Mean Squared Error (MSE): 174110.6857142857
Mean Squared Error (MSE): 81525.34285714285
Mean Squared Error (MSE): 69398.74285714286
Mean Squared Error (MSE): 68678.25714285714
